# Federated Deep Model with Flex  for Anomaly Detection with Time Series

First we do all the imports needed.

In [2]:
from flexanomalies.models import DeepCNN_LSTM
from flexanomalies.utils.load_data import load_and_split_csv, federate_data
from flexanomalies.pool.decorators_autoencoder import build_server_model_ae, copy_model_to_clients_ae,train_ae, aggregate_ae, set_aggregated_weights_ae, weights_collector_ae
from flexanomalies.utils.save_results import save_experiments_results
from flex.pool import FlexPool
from flexanomalies.utils.metrics import *
import pandas as pd 


2023-12-11 01:03:09.144388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-11 01:03:09.256866: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-11 01:03:09.260932: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-12-11 01:03:09.260949: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [3]:
model_params = {
    "epochs": 5,
    "input_dim":41,
    "batch_size": 32,
    "filters_cnn": [32,32],
    "units_lstm": [32,32],
    "kernel_size": [5,5],
    "hidden_act": ["relu","relu"],
    "w_size":30,
    "n_pred": 1,
    "contamination": 0.1
}

file_path = "../flex-anomalies/flexanomalies/datasets/data/corrected.gz"
split_test = 0.3

In [4]:


X_train, X_test, l_train, l_test = load_and_split_csv(file_path,model_params["input_dim"],split_test)
for i in range(len(l_train)):
    if l_train[i] == "normal.":
        l_train[i] = 0
    else:
         l_train[i] = 1   

for i in range(len(l_test)):
    if l_test[i] == "normal.":
        l_test[i] = 0
    else:
         l_test[i] = 1   

In [11]:
def create_windows (w_size,n_pred):   # Number of predictions into the future
    X_train_windows = []
    y_train_windows = []
    X_test_windows = []
    y_test_windows = []  
    
   
    for i in range(w_size, len(X_train) - n_pred +1,n_pred):
        X_train_windows.append(X_train[i - w_size:i, :])                        
        
        y_train_windows.append(X_train[i:i + n_pred, :])  

       # y_train_windows.append(y_train[i:i + n_pred].reshape(-1,1))        
        

    #Test    
    for i in range(w_size, len(X_test) - n_pred +1,n_pred):
        X_test_windows.append(X_test[i - w_size:i, :])                        
        
        y_test_windows.append(X_test[i:i + n_pred, :])  

        #y_test_windows.append(y_test[i:i + n_pred].reshape(-1,1))   
       
    print('X_train shape == {}.'.format(np.array(X_train_windows).shape))
    
    print('y_train shape == {}.'.format(np.array(y_train_windows).shape))

    print('X_test shape == {}.'.format(np.array(X_test_windows).shape))
   
    print('y_test shape == {}.'.format(np.array(y_test_windows).shape))
   

    return np.array(X_train_windows, np.array(y_train_windows), np.array(X_test_windows), np.array(y_test_windows)


In [12]:
X_train_windows, y_train_windows, X_test_windows,y_test_windows  = create_windows(30,10)

X_train shape == (21768, 30, 41).
y_train shape == (21768, 10, 41).
X_test shape == (9327, 30, 41).
y_test shape == (9327, 10, 41).


AttributeError: 'list' object has no attribute 'astype'

In [7]:

model = DeepCNN_LSTM(**model_params)


2023-12-11 01:03:13.175229: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-12-11 01:03:13.175251: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-12-11 01:03:13.175271: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (beatriz-IdeaPad-3-15ITL6): /proc/driver/nvidia/version does not exist
2023-12-11 01:03:13.175531: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


relu 32 5
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 26, 32)            6592      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 13, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 9, 32)             5152      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 4, 32)            0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 4, 32)             8320      
                                                                 
 lstm_1 (LSTM)               (None, 4, 32)    

In [8]:
flex_dataset = federate_data(5, X_train_windows, y_train_windows)


In [9]:
pool = FlexPool.client_server_pool(
        fed_dataset=flex_dataset,
        server_id="cnn_lstm_server",
        init_func=build_server_model_ae,
        model=model,
    )
pool.clients._data["client1"]

Dataset(X_data=len:4353
is_generator:False
iterable:[[[0 'tcp' 'private' ... 0.0 1.0 1.0]
  [0 'tcp' 'http' ... 0.0 0.0 0.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  ...
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]]

 [[0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'tcp' 'private' ... 0.0 1.0 1.0]
  [0 'tcp' 'private' ... 1.0 0.0 0.0]
  ...
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'tcp' 'private' ... 0.0 1.0 1.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]]

 [[0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'udp' 'domain_u' ... 0.0 0.0 0.0]
  ...
  [0 'udp' 'private' ... 0.0 0.0 0.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]]

 ...

 [[0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'tcp' 'http' ... 0.0 0.0 0.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  ...
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [0 'icmp' 'ecr_i' ... 0.0 0.0 0.0]
  [4 'tcp' 'pop_3' ... 0.0 0.0 0.0]]

 [[0 'icmp' 'ecr_i' 

In [10]:
for i in range(3):
    print(f"\nRunning round: {i}\n")
    pool.servers.map(copy_model_to_clients_ae, pool.clients)
    pool.clients.map(train_ae)
    pool.aggregators.map(weights_collector_ae, pool.clients)
    pool.aggregators.map(aggregate_ae)
    pool.aggregators.map(set_aggregated_weights_ae, pool.servers)
output_model = pool.servers._models["cnn_lstm_server"]["model"]
output_model.evaluate(X_test_windows,l_test)


Running round: 0

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......conv1d
.........vars
............0
............1
......conv1d_1
.........vars
............0
............1
......dense
.........vars
............0
............1
......flatten
.........vars
......lstm
.........cell
............vars
...............0
...............1
...............2
.........vars
......lstm_1
.........cell
............vars
...............0
...............1
...............2
.........vars
......max_pooling1d
.........vars
......max_pooling1d_1
.........vars
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-12-11 01:03:14         4242
variables.h5                                   2023-12-11 01:03:14       162904
metadata.json                                  2023-12-11 01:03:14           64
Keras model archive loading:
File 

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [ ]:
output_model.result_metrics_

In [ ]:
save_experiments_results(
    'cnn_lstm',
    output_model,
    'test_cnn_lstm_notebook',
    model_params,
    'shuttle.mat',
    5,
    3,
    0.3,
)